# Environement Setup

In [4]:
# @title 1. 🛠️ Installazione Ambiente (PyTorch 2.4.0)
import os
import sys

print("🔄 Preparazione ambiente in corso... (richiederà 2-3 minuti)")

# 1. Disinstallazione versioni incompatibili
os.system("pip uninstall -y torch torchvision torchaudio torch-scatter torch-sparse torch-cluster torch-geometric")

# 2. Installazione PyTorch 2.4.0 (Versione richiesta dal README)
# Usiamo CUDA 12.1 che è stabile su Colab
cmd_torch = "pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121"
os.system(cmd_torch)

# 3. Installazione librerie Grafiche (PyG) specifiche per Torch 2.4.0
pyg_url = "https://data.pyg.org/whl/torch-2.4.0+cu121.html"
libs = ["torch-scatter", "torch-sparse", "torch-cluster", "torch-spline-conv"]
cmd_pyg = f"pip install {' '.join(libs)} -f {pyg_url}"
os.system(cmd_pyg)

# 4. Altre dipendenze
os.system("pip install torch-geometric hydra-core omegaconf submitit pandas")

print("✅ Installazione completata.")
print("⚠️ IMPORTANTE: Ora vai nel menu in alto: 'Runtime' > 'Restart Session' (Riavvia Sessione) prima di eseguire la cella successiva!")

⬇️ Installazione PyTorch 2.4.0...
⬇️ Installazione dipendenze PyG...
✅ Installazione completata come da README.
⚠️ ORA RIAVVIA IL RUNTIME (Runtime > Restart Session)!


In [1]:
# @title 2. 📂 Setup HiERO e Download Pesi Pre-addestrati
import os

# 1. Clone del repository
if not os.path.exists("HiERO"):
    print("⬇️ Clonazione repository HiERO...")
    os.system("git clone --recursive https://github.com/sapeirone/HiERO.git")

os.chdir("/content/HiERO")

# 2. Creazione struttura cartelle
dirs = [
    "checkpoints",
    "pretrained",
    "data/ego4d/raw/annotations",
    "data/ego4d/raw/features/egovlp"
]
for d in dirs:
    os.makedirs(d, exist_ok=True)

# 3. Download del modello HiERO già addestrato (Backbone EgoVLP)
if not os.path.exists("checkpoints/hiero_egovlp.pth"):
    print("⬇️ Scaricamento Modello Pre-addestrato (hiero_egovlp.pth)...")
    os.system("wget -O checkpoints/hiero_egovlp.pth https://www.sapeirone.it/data/hiero/ckpt/hiero_egovlp.pth")

# 4. Download componenti testuali (richiesti dal codice)
print("⬇️ Scaricamento Encoder Testuale...")
os.system("wget -q -O pretrained/egovlp_text.pth https://www.sapeirone.it/data/hiero/pretrained/egovlp_text.pth")
os.system("wget -q -O pretrained/egovlp_txt_proj.pth https://www.sapeirone.it/data/hiero/pretrained/egovlp_txt_proj.pth")

# 5. Download Annotazioni necessarie (EgoMCQ/EgoClip)
print("⬇️ Scaricamento Annotazioni...")
os.system("gdown --id 1-aaDu_Gi-Y2sQI_2rsI2D1zvQBJnHpXl -O data/ego4d/raw/annotations/egoclip.csv")
os.system("gdown --id 1-5iRYf4BCHmj4MYQYFRMY4bhsWJUN3rW -O data/ego4d/raw/annotations/egomcq.json")

print("✅ Setup completato. Cartelle pronte.")

⬇️ Scaricamento Modello Pre-addestrato (hiero_egovlp.pth)...
⬇️ Scaricamento Encoder Testuale...
⬇️ Scaricamento Annotazioni...
✅ Setup completato. Cartelle pronte.


In [30]:
import os
import shutil
import torch
import glob
import sys
import numpy as np
from tqdm import tqdm
from omegaconf import OmegaConf

# ==========================================
# 1. SETUP PERCORSI CORRETTI
# ==========================================
# 🛑 PUNTO CRITICO: Qui mettiamo il percorso che mi hai appena dato
USER_ACTUAL_PATH = "/content/data/egovlp/egovlp_test"

HIERO_ROOT = "/content/HiERO"
OUTPUT_DIR = "/content/output_embeddings"

if HIERO_ROOT not in sys.path:
    sys.path.append(HIERO_ROOT)
os.chdir(HIERO_ROOT)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📂 Cartella Input Dati: {USER_ACTUAL_PATH}")
print(f"📂 Cartella Output: {OUTPUT_DIR}")

# ==========================================
# 2. CONFIGURAZIONE MODELLO
# ==========================================
print("\n⚙️ Configurazione Modello...")
try:
    from models.hiero import HiERO

    # Configurazione Layer
    conv_config = OmegaConf.create({
        "_target_": "models.conv.tdgc.TDGC",
        "in_channels": 768, "out_channels": 768, "bias": True,
        "pos_hidden_channels": 256, "aggr": "mean", "pos_act": "tanh",
        "use_signed_temporal_distance": True, "use_pos_attr": True
    })

    # Istanziazione Modello
    model = HiERO(
        input_size=256, hidden_size=768, conv=conv_config,
        depth=3, n_clusters=8, input_projection=True, dropout=0.1,
        pool="batch_subsampling"
    )

    # Caricamento Pesi
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ckpt_path = os.path.join(HIERO_ROOT, "checkpoints/hiero_egovlp.pth")

    if os.path.exists(ckpt_path):
        checkpoint = torch.load(ckpt_path, map_location="cpu")
        state_dict = checkpoint['model'] if 'model' in checkpoint else checkpoint
        model.load_state_dict(state_dict, strict=False)
        model = model.to(device).eval()
        print(f"✅ Modello pronto su {device}.")
    else:
        raise FileNotFoundError(f"Modello mancante in {ckpt_path}")

    # ==========================================
    # 3. RICERCA E ELABORAZIONE (PTH + NPZ)
    # ==========================================
    print(f"\n🔍 Cerco file .pth e .npz in {USER_ACTUAL_PATH}...")

    # Cerchiamo sia .pth che .npz
    files_pth = glob.glob(f"{USER_ACTUAL_PATH}/*.pth")
    files_npz = glob.glob(f"{USER_ACTUAL_PATH}/*.npz")
    files = files_pth + files_npz

    print(f"📂 Trovati {len(files)} file totali ({len(files_pth)} pth, {len(files_npz)} npz).")

    count = 0
    if len(files) == 0:
        print("❌ ERRORE: La cartella sembra vuota! Controlla se il percorso è esatto.")
    else:
        with torch.no_grad():
            for fpath in tqdm(files):
                filename = os.path.basename(fpath)

                # Salta il modello se per caso è lì dentro
                if "hiero_egovlp.pth" in filename: continue

                # Nome file output (sempre .pth per compatibilità PyTorch)
                out_name = filename.replace('.npz', '.pth')
                save_path = os.path.join(OUTPUT_DIR, out_name)

                try:
                    feat = None

                    # --- CARICAMENTO ---
                    if fpath.endswith('.npz'):
                        # Logica per .npz
                        with np.load(fpath, allow_pickle=True) as data:
                            keys = list(data.keys())
                            # Cerca la chiave giusta
                            if 'features' in keys: feat_np = data['features']
                            elif 'arr_0' in keys: feat_np = data['arr_0']
                            else: feat_np = data[keys[0]] # Fallback sulla prima

                            # Converti a Tensor
                            if isinstance(feat_np, np.ndarray):
                                feat = torch.from_numpy(feat_np).float()
                            else:
                                feat = torch.tensor(feat_np).float()
                    else:
                        # Logica per .pth
                        feat = torch.load(fpath, map_location=device)
                        if isinstance(feat, dict):
                            for k in ['features', 'feats', 'video_features']:
                                if k in feat:
                                    feat = feat[k]; break
                            else:
                                if len(feat)>0: feat = list(feat.values())[0]

                    # --- CONTROLLI ---
                    if not isinstance(feat, torch.Tensor):
                        print(f"⚠️ {filename}: Formato dati non valido.")
                        continue

                    feat = feat.to(device)
                    # Aggiungi dimensione batch se manca (Time, Feat) -> (1, Time, Feat)
                    if feat.dim() == 2: feat = feat.unsqueeze(0)

                    B, T, F_dim = feat.shape
                    if T == 0: continue # Video vuoto

                    # --- PREPARAZIONE DATI HiERO ---
                    from torch_geometric.data import Data
                    pos = torch.arange(T, device=device, dtype=torch.float).repeat(B)
                    batch = torch.arange(B, device=device).repeat_interleave(T)
                    indices = torch.arange(T, device=device).repeat(B)
                    x_flat = feat.reshape(-1, F_dim)

                    data_obj = Data(x=x_flat, pos=pos, batch=batch, indices=indices)
                    data_obj.mask = torch.ones(x_flat.shape[0], dtype=torch.bool, device=device)

                    # --- INFERENZA ---
                    out_batch = model(data_obj)
                    final_emb = out_batch.x

                    # --- SALVATAGGIO ---
                    torch.save(final_emb.cpu(), save_path)
                    count += 1

                except Exception as e:

                    print(f"⚠️ Errore su {filename}: {e}")

        print(f"\n✅ OPERAZIONE COMPLETATA!")
        print(f"   Salvato {count} file in: {OUTPUT_DIR}")
        print("   (Controlla la cartella 'output_embeddings' a sinistra per scaricarli)")

except Exception as e:
    print(f"\n❌ ERRORE IRRECUPERABILE: {e}")

📂 Cartella Input Dati: /content/data/egovlp/egovlp_test
📂 Cartella Output: /content/output_embeddings

⚙️ Configurazione Modello...


/tmp/ipython-input-1357212630.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location="cpu")


✅ Modello pronto su cuda.

🔍 Cerco file .pth e .npz in /content/data/egovlp/egovlp_test...
📂 Trovati 10 file totali (0 pth, 10 npz).


 10%|█         | 1/10 [00:00<00:01,  6.20it/s]

⚠️ Errore su 3_18_360p_224.mp4_1s_1s.npz: 2
⚠️ Errore su 1_14_360p_224.mp4_1s_1s.npz: 2


100%|██████████| 10/10 [00:00<00:00, 27.88it/s]

⚠️ Errore su 3_5_360p_224.mp4_1s_1s.npz: 2
⚠️ Errore su 3_13_360p_224.mp4_1s_1s.npz: 2
⚠️ Errore su 3_12_360p_224.mp4_1s_1s.npz: 2
⚠️ Errore su 1_32_360p_224.mp4_1s_1s.npz: 2
⚠️ Errore su 1_10_360p_224.mp4_1s_1s.npz: 2
⚠️ Errore su 3_14_360p_224.mp4_1s_1s.npz: 2
⚠️ Errore su 1_19_360p_224.mp4_1s_1s.npz: 2
⚠️ Errore su 2_28_360p_224.mp4_1s_1s.npz: 2

✅ OPERAZIONE COMPLETATA!
   Salvato 0 file in: /content/output_embeddings
   (Controlla la cartella 'output_embeddings' a sinistra per scaricarli)
